# Convert polygons with categorical values to gridded binary layers
Given a shapefile with polygons and a category field, ouutput a COG file with a binary mask for each category.
Steps:
1. Transfer categorical values from the vector dataset to the vector grid covering the study area.
2. export vector grid to GeoTiff
3. convert GeoTiff to COG

In [99]:
grid_path = 'N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f00_fishnet/fishnet_10m_2169.shp'
aoi_path = 'N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f06_POIs/AOIs_OSM_2169.gpkg'
poi_path = 'N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f06_POIs/POIs_OSM_2169.gpkg'
output_path = 'N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f02_raster_10m_collection/aoi_osm_2024_10m_b4.tif'


In [141]:
import geopandas as gpd
gdf_aoi = gpd.read_file(aoi_path)
gdf_poi = gpd.read_file(poi_path)
grid = gpd.read_file(grid_path)
union_aoi = gpd.overlay(grid, gdf_aoi, how='union')[['id', 'aoi_name', 'geometry']]
union_poi = gpd.sjoin(grid, gdf_poi, how='inner', predicate='contains')[['id', 'poi_name', 'geometry']]

In [146]:
import pandas as pd
union_gdf = gpd.GeoDataFrame(pd.concat([union_aoi, union_poi], ignore_index=True), crs=union_aoi.crs)

In [148]:
import rasterio
from rasterio.features import rasterize
from rasterio.transform import from_bounds
from shapely import union

# parameters
xmin = 72850.0
ymin = 69590.0
xmax = 82570.0
ymax = 80030.0

projection ='epsg:2169'

pixel_size = 10

width = int((xmax - xmin) / pixel_size)
height = int((ymax - ymin) / pixel_size)
transform = from_bounds(xmin, ymin, xmax, ymax, width, height)
n_categories = 4
raster_cube = []
# function
for cat in range(1, n_categories+1):
    cells = union_gdf[(union_gdf.aoi_name == cat) | (union_gdf.poi_name == cat)]
    # Create an empty array to hold rasterized values
    raster_array = rasterize(
        [(geom, 1) for geom in cells.geometry],
        out_shape=(height, width),
        transform=transform,
        fill=0,
        all_touched=True,  # or True
        dtype=rasterio.uint8
    )
    raster_cube.append(raster_array)



In [150]:
import numpy as np
raster_cube_np = np.array(raster_cube)
with rasterio.open(output_path,
                    "w", driver="GTiff", 
            height=raster_array.shape[0], 
            width=raster_array.shape[1], 
            count=4, 
            dtype=raster_array.dtype, 
            crs=cells.crs, 
            transform=transform
            ) as dst:
    dst.write(raster_cube_np[0,:,:].astype(rasterio.uint8),1)
    dst.write(raster_cube_np[1,:,:].astype(rasterio.uint8),2)
    dst.write(raster_cube_np[2,:,:].astype(rasterio.uint8),3)
    dst.write(raster_cube_np[3,:,:].astype(rasterio.uint8),4)

In OSGEO console, run:
````
gdal_translate -a_srs EPSG:2169 -a_nodata 99.0 -of COG N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f02_raster_10m_collection/aoi_osm_2024_10m_b4.tif N:/C2205_FAIRiCUBE/f02_data/d060_data_LUXEMBOURG/f01_Lux_neopyhtes/f02_luxembourg_data/f02_raster_10m_collection/aoi_osm_2024_10m_b4_cog.tif
```